In [78]:
# Optional: Create the CSV file using Python
content = """
order_id,region,product,quantity,price
4001,East,Keyboard,2,1500
4002,West,Mouse,,500
4003,East,Monitor,1,12000
4004,South,Keyboard,1,1500
4005,West,Monitor,2,12000
4005,West,Monitor,2,12000
"""

with open("sales_raw.csv", "w") as file:
    file.write(content)

print("sales_raw.csv created successfully!")


sales_raw.csv created successfully!


In [80]:
import pandas as pd
df = pd.read_csv("sales_raw.csv")


In [81]:
print("=== DATASET INSPECTION ===")
print(f"Shape: {df.shape}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")
print(f"\nFirst few rows:\n{df.head()}")
print(f"\nDuplicate rows: {df.duplicated().sum()}")


=== DATASET INSPECTION ===
Shape: (6, 5)

Data types:
order_id      int64
region       object
product      object
quantity    float64
price         int64
dtype: object

Missing values:
order_id    0
region      0
product     0
quantity    1
price       0
dtype: int64

First few rows:
   order_id region   product  quantity  price
0      4001   East  Keyboard       2.0   1500
1      4002   West     Mouse       NaN    500
2      4003   East   Monitor       1.0  12000
3      4004  South  Keyboard       1.0   1500
4      4005   West   Monitor       2.0  12000

Duplicate rows: 1


In [82]:
print(f"Rows before removing duplicates: {len(df)}")
df_clean = df.drop_duplicates()
print(f"Rows after removing duplicates: {len(df_clean)}")


Rows before removing duplicates: 6
Rows after removing duplicates: 5


In [83]:
print(f"\nRows before dropping missing quantity: {len(df_clean)}")
df_clean = df_clean.dropna(subset=["quantity"])
print(f"Rows after dropping missing quantity: {len(df_clean)}")



Rows before dropping missing quantity: 5
Rows after dropping missing quantity: 4


In [84]:
df_clean["quantity"] = pd.to_numeric(df_clean["quantity"], errors='coerce')
df_clean["price"] = pd.to_numeric(df_clean["price"], errors='coerce')

print("\n=== DATA TYPES AFTER CLEANING ===")
print(df_clean.dtypes)



=== DATA TYPES AFTER CLEANING ===
order_id      int64
region       object
product      object
quantity    float64
price         int64
dtype: object


In [85]:
print("\n=== CLEANING VERIFICATION ===")
print(f"Missing values:\n{df_clean.isnull().sum()}")
print(f"Duplicates: {df_clean.duplicated().sum()}")
print(f"\nCleaned dataset:\n{df_clean}")



=== CLEANING VERIFICATION ===
Missing values:
order_id    0
region      0
product     0
quantity    0
price       0
dtype: int64
Duplicates: 0

Cleaned dataset:
   order_id region   product  quantity  price
0      4001   East  Keyboard       2.0   1500
2      4003   East   Monitor       1.0  12000
3      4004  South  Keyboard       1.0   1500
4      4005   West   Monitor       2.0  12000


In [86]:
df_clean["revenue"] = df_clean["quantity"] * df_clean["price"]
print("Dataset with revenue:")
print(df_clean)


Dataset with revenue:
   order_id region   product  quantity  price  revenue
0      4001   East  Keyboard       2.0   1500   3000.0
2      4003   East   Monitor       1.0  12000  12000.0
3      4004  South  Keyboard       1.0   1500   1500.0
4      4005   West   Monitor       2.0  12000  24000.0


In [87]:
revenue_by_product = df_clean.groupby("product")["revenue"].sum().reset_index()
revenue_by_product.columns = ["product", "total_revenue"]
revenue_by_product = revenue_by_product.sort_values("total_revenue", ascending=False)

print("\n=== REVENUE BY PRODUCT ===")
print(revenue_by_product)



=== REVENUE BY PRODUCT ===
    product  total_revenue
1   Monitor        36000.0
0  Keyboard         4500.0


In [88]:
revenue_by_region = df_clean.groupby("region")["revenue"].sum().reset_index()
revenue_by_region.columns = ["region", "total_revenue"]
revenue_by_region = revenue_by_region.sort_values("total_revenue", ascending=False)

print("\n=== REVENUE BY REGION ===")
print(revenue_by_region)



=== REVENUE BY REGION ===
  region  total_revenue
2   West        24000.0
0   East        15000.0
1  South         1500.0


In [89]:
df_clean.to_csv("clean_sales.csv", index=False)
print("Saved: clean_sales.csv")


Saved: clean_sales.csv


In [90]:
revenue_by_product.to_csv("revenue_by_product.csv", index=False)
print("Saved: revenue_by_product.csv")


Saved: revenue_by_product.csv


In [91]:
revenue_by_region.to_csv("revenue_by_region.csv", index=False)
print("Saved: revenue_by_region.csv")


Saved: revenue_by_region.csv


In [92]:
print("\n=== SUMMARY ===")
print(f"Total records cleaned: {len(df_clean)}")
print(f"Total revenue: {df_clean['revenue'].sum():,.2f}")
print(f"Products analyzed: {len(revenue_by_product)}")
print(f"Regions analyzed: {len(revenue_by_region)}")



=== SUMMARY ===
Total records cleaned: 4
Total revenue: 40,500.00
Products analyzed: 2
Regions analyzed: 3


In [93]:
def clean_and_aggregate_sales(input_file):
    """
    Clean sales data and generate revenue summaries.
    """
    # Load data
    df = pd.read_csv(input_file)
    print(f"Loaded {len(df)} records")
    
    # Clean data
    df_clean = df.drop_duplicates()
    df_clean = df_clean.dropna(subset=["quantity"])
    df_clean["quantity"] = pd.to_numeric(df_clean["quantity"], errors='coerce')
    df_clean["price"] = pd.to_numeric(df_clean["price"], errors='coerce')
    
    # Calculate revenue
    df_clean["revenue"] = df_clean["quantity"] * df_clean["price"]
    
    # Aggregations
    revenue_by_product = df_clean.groupby("product")["revenue"].sum().reset_index()
    revenue_by_product.columns = ["product", "total_revenue"]
    revenue_by_product = revenue_by_product.sort_values("total_revenue", ascending=False)
    
    revenue_by_region = df_clean.groupby("region")["revenue"].sum().reset_index()
    revenue_by_region.columns = ["region", "total_revenue"]
    revenue_by_region = revenue_by_region.sort_values("total_revenue", ascending=False)
    
    # Save outputs
    df_clean.to_csv("clean_sales.csv", index=False)
    revenue_by_product.to_csv("revenue_by_product.csv", index=False)
    revenue_by_region.to_csv("revenue_by_region.csv", index=False)
    
    print("Cleaning and aggregation complete!")
    print(f"  - Clean records: {len(df_clean)}")
    print(f"  - Total revenue: {df_clean['revenue'].sum():,.2f}")
    
    return df_clean, revenue_by_product, revenue_by_region


In [94]:
clean_df, product_rev, region_rev = clean_and_aggregate_sales("sales_raw.csv")


Loaded 6 records
Cleaning and aggregation complete!
  - Clean records: 4
  - Total revenue: 40,500.00
